In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from domain_model import Scenario, StateVariable
from simulation import ACC, acc_lead_braking_pars, ACCHDM, acc_hdm_lead_braking_pars, \
    hdm_lead_braking_pars, SimulationLeadBraking, HDM, EIDM, eidm_lead_braking_pars, \
    IDMPlus, IDMParameters, HDMParameters, SimulationApproaching, hdm_approaching_pars, \
    acc_approaching_pars, acc_hdm_approaching_pars, SimulationCutIn, hdm_cutin_pars, \
    acc_cutin_pars, acc_hdm_cutin_pars
from stats import KDE, kde_from_file
from databaseemulator import DataBaseEmulator
%matplotlib inline

# Test driver model, ACC, and CACC

In [ ]:
s = SimulationLeadBraking(follower=HDM(), follower_parameters=hdm_lead_braking_pars)

In [ ]:
np.random.seed(0)
s.simulation(dict(v0=20, amean=2.5, dv=20), plot=True, seed=0)

In [ ]:
s2 = SimulationLeadBraking(follower=EIDM(), follower_parameters=eidm_lead_braking_pars)
s2.simulation(dict(v0=20, amean=2, dv=20), plot=True, seed=0)

In [ ]:
s2 = SimulationLeadBraking(follower=ACC(), follower_parameters=acc_lead_braking_pars)
s2.simulation(dict(v0=5, amean=1, dv=4), plot=True, seed=0)

In [ ]:
s3 = SimulationLeadBraking(follower=ACCHDM(), follower_parameters=acc_hdm_lead_braking_pars)
s3.simulation(dict(v0=20, amean=6, dv=18), plot=True, seed=0)

# Approaching vehicle

In [ ]:
s1 = SimulationApproaching(follower=HDM(), follower_parameters=hdm_approaching_pars)
s1.simulation(dict(ratio_vtar_vego=0, vego=20), plot=True, seed=0)

In [ ]:
s2 = SimulationApproaching(follower=ACC(), follower_parameters=acc_approaching_pars)
s2.simulation(dict(ratio_vtar_vego=0, vego=20), plot=True, seed=0)

In [ ]:
s3 = SimulationApproaching(follower=ACCHDM(), follower_parameters=acc_hdm_approaching_pars)
s3.simulation(dict(ratio_vtar_vego=0.1, vego=15), plot=True, seed=0)

# Cut-in

In [ ]:
s1 = SimulationCutIn(follower=HDM(), follower_parameters=hdm_cutin_pars)
s1.simulation(dict(vego=40, ratio_vtar_vego=0.8, dinit=20), plot=True, seed=0)

In [ ]:
s2 = SimulationCutIn(follower=ACC(), follower_parameters=acc_cutin_pars)
s2.simulation(dict(vego=20, ratio_vtar_vego=0.8, dinit=20), plot=True, seed=0)

In [ ]:
s3 = SimulationCutIn(follower=ACCHDM(), follower_parameters=acc_hdm_cutin_pars)
s3.simulation(dict(vego=20, ratio_vtar_vego=0.7, dinit=20), plot=True, seed=0)

In [ ]:
d = DataBaseEmulator(os.path.join("data", "5_scenarios", "cut_in_scenarios.json"))

In [ ]:
s = d.get_ordered_item("scenario", 0)

In [ ]:
x = []
for i in range(len(d.collections["scenario"])):
    s = d.get_ordered_item("scenario", i)
    t = np.linspace(s.time["start"], s.time["end"], 100)
    y = s.get_state(s.get_actor_by_name("target vehicle"), StateVariable.LAT_TARGET, t)
    y = np.abs(y)
    t = t[np.gradient(y) < 0]
    y = y[np.gradient(y) < 0]
    time_cutin = np.interp(2, y, t)
    vtarget, dinit = s.get_state(s.get_actor_by_name("target vehicle"), 
                                 StateVariable.LON_TARGET, time_cutin)
    vego = s.get_state(s.get_actor_by_name("ego vehicle"), StateVariable.SPEED, time_cutin)
    if 0 < vtarget < vego:
        x.append([vego, dinit, vtarget/vego])
x = np.array(x)

In [ ]:
plt.plot(x[:, 0], x[:, 1], '.')

In [ ]:
np.sort(x[:, 1])

In [ ]:
plt.plot(x[:, 2], x[:, 1], '.')

In [ ]:
tt = np.linspace(1.5, 2, 51)
c = np.zeros(51)
for i, t in enumerate(tt):
    c[i] = np.sum(np.logical_and(x[:, 0] > t, x[:, 1] < t))

In [ ]:
plt.plot(tt, c)

In [ ]:
len(x)

In [ ]:
dfdsfsafdf

In [ ]:
%debug

In [ ]:
d = DataBaseEmulator(os.path.join("data", "5_scenarios", "ego_braking.json"))
print(len(d.collections["scenario"]))
d = DataBaseEmulator(os.path.join("data", "5_scenarios", "lead_braking.json"))
print(len(d.collections["scenario"]))  # 1310
d = DataBaseEmulator(os.path.join("data", "5_scenarios", "approaching_vehicle.json"))
print(len(d.collections["scenario"]))  # 655

In [ ]:
d = DataBaseEmulator(os.path.join("data", "5_scenarios", "ego_braking.json"))
print(len(d.collections["scenario"]))
d = DataBaseEmulator(os.path.join("data", "5_scenarios", "lead_braking.json"))
print(len(d.collections["scenario"]))  # 1310
d = DataBaseEmulator(os.path.join("data", "5_scenarios", "approaching_vehicle.json"))
print(len(d.collections["scenario"]))  # 655

In [ ]:
def parameters_lead_braking() -> np.ndarray:
    """ Return the parameters for lead vehicle braking.

    The parameters are:
    1. Initial speed
    2. Average deceleration
    3. Speed difference (positive means braking, i.e., vstart-vend)

    :return: The numpy array with the parameters.
    """
    data = DataBaseEmulator(os.path.join("data", "5_scenarios", "lead_braking - Copy.json"))
    pars = []
    for i in range(len(data.collections["scenario"])):
        scenario = data.get_ordered_item("scenario", i)
        vstart, vdiff, amean = 0, 0, 0
        for activity in scenario.activities:
            if activity.name == "deceleration target":
                vstart, vend = activity.get_state(time=[activity.tstart, activity.tend])[0]
                vdiff = vstart-vend
                amean = vdiff/(activity.tend-activity.tstart)
                break

        if vstart > 0 and vdiff > 0 and amean > 0:
            pars.append([vstart, amean, vdiff])
    return np.array(pars)

In [ ]:
x = parameters_lead_braking()

In [ ]:
y = parameters_lead_braking()

In [ ]:
i = 0
j = 1
plt.plot(y[:, i], y[:, j], 's')
plt.plot(x[:, i], x[:, j], '.')

# Approaching vehicle scenario

In [ ]:
import glob
import os
from activity_detector import LeadVehicle, LateralStateTarget, LongitudinalStateTarget, \
    TargetNear
from data_handler import DataHandler
from create_ngrams import FIELDNAMES_EGO, METADATA_EGO, FIELDNAMES_TARGET, METADATA_TARGET
from ngram import NGram
from ngram_search import find_sequence

In [ ]:
d = DataBaseEmulator(os.path.join("data", "5_scenarios", "approaching_vehicle.json"))

In [ ]:
s.get_actor_by_name("target vehicle")

In [ ]:
vtars = []
vegos = []
for i in range(len(d.collections["scenario"])):
    s = d.get_ordered_item("scenario", i)
    vego = s.get_state(s.actors[0], StateVariable.SPEED, s.time["start"])
    vtar = s.get_state(s.actors[1], StateVariable.LON_TARGET, s.time["start"])[0]
    if vego > vtar:
        vtars.append(vtar)
        vegos.append(vego)
vtars = np.array(vtars)
vegos = np.array(vegos)

In [ ]:
plt.plot(vegos, vtars/vegos, '.')

In [ ]:
t = np.linspace(s.time["start"], s.time["end"], 100)
vego = s.get_state(s.actors[0], StateVariable.SPEED, t)
vtar = s.get_state(s.actors[1], StateVariable.LON_TARGET, t)[:, 0]
dtar = s.get_state(s.actors[1], StateVariable.LON_TARGET, t)[:, 1]

In [ ]:
plt.plot(t, vego, label="Ego")
plt.plot(t, vtar, label="Target")
plt.legend()

In [ ]:
vtar[0]

In [ ]:
plt.plot(t, np.cumsum(vtar-vego)*np.gradient(t)-(np.cumsum(vtar-vego)*np.gradient(t))[0])
plt.plot(t, dtar-dtar[0])

In [ ]:
np.gradient(t)

In [ ]:
np.cumsum(vtar-vego)*np.gradient(t)